In [46]:
from drn_interactions.config import ExperimentInfo, Config
from drn_interactions.transforms import SpikesHandler
from drn_interactions.io import load_derived_generic
from drn_interactions.stats import mannwhitneyu_plusplus
import numpy as np
import pandas as pd
from scipy.stats import zscore, zmap
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [47]:
def load_drug_data(
    block="chal", 
    t_start=-600, 
    t_stop=1200, 
    bin_width=1,
    sessions=None,
    merge_neuron_types=False,
    group_names=None,
    ):
    df_binned = SpikesHandler(
        block=block,
        bin_width=bin_width,
        session_names=sessions,
        t_start=t_start,
        t_stop=t_stop,
    ).binned
    df_binned["block"] = np.where(df_binned["bin"] < 0, "pre", "post")
    neuron_types = load_derived_generic("neuron_types.csv")[["neuron_id", "neuron_type", "session_name", "group_name"]]
    df_binned = df_binned.merge(neuron_types)
    if group_names is not None:
        df_binned = df_binned.query("group_name in @group_names")
    return df_binned



In [54]:
neuron_types = load_derived_generic("neuron_types.csv")
df_chal =load_drug_data()
df_chal["zcounts"] = df_chal.groupby("neuron_id")["counts"].transform(lambda x: zmap(x, x[:600]))
df_chal = df_chal.dropna()
df_chal.to_csv(Config.derived_data_dir / "chal_binned.csv", index=False)

In [55]:
def mwu(df):
    pre = df.query("block == 'pre'")["zcounts"].values
    post = df.query("block == 'post'")["zcounts"].values
    return mannwhitneyu_plusplus(pre, post, names=("pre", "post"))

df_responders = (
    df_chal
    .groupby("neuron_id")
    .apply(mwu)
    .reset_index()
    .merge(neuron_types)
)
df_responders.to_csv(Config.derived_data_dir / "chal_responders.csv", index=False)


In [56]:
%%R

suppressMessages(require(lme4))
suppressMessages(require(lmerTest))
suppressMessages(require(emmeans))
suppressMessages(require(tidyverse))

p <- "C:\\Users\\roryl\\repos\\DRN Interactions\\data\\derived\\chal_binned.csv"

df <- suppressMessages(read_csv(p)) %>%
  mutate(
    neuron_type=factor(neuron_type, levels=c("SR", "SIR", "FF")),
    drug=factor(
      if_else(str_detect(group_name ,"sal"), "SAL", "CIT"), 
      levels=c("SAL", "CIT")),
    block=factor(block, levels=c("pre", "post"))
    )

# Model
mod <- lm(zcounts ~ block * drug *neuron_type, data=df)

# ANOVA
print(anova(mod))

Analysis of Variance Table

Response: zcounts
                           Df  Sum Sq Mean Sq F value    Pr(>F)    
block                       1    3162  3162.4  851.27 < 2.2e-16 ***
drug                        1   12934 12934.1 3481.65 < 2.2e-16 ***
neuron_type                 2   37390 18695.2 5032.46 < 2.2e-16 ***
block:drug                  1    6472  6472.4 1742.28 < 2.2e-16 ***
block:neuron_type           2   18711  9355.4 2518.33 < 2.2e-16 ***
drug:neuron_type            2   21637 10818.3 2912.11 < 2.2e-16 ***
block:drug:neuron_type      2   10827  5413.6 1457.27 < 2.2e-16 ***
Residuals              647628 2405884     3.7                      
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1


In [57]:
%%R

# EMMEANS
em <- emmeans(mod, ~ block*drug*neuron_type)

print(em)


 block drug neuron_type  emmean      SE     df lower.CL upper.CL
 pre   SAL  SR           0.0000 0.00812 647628  -0.0159   0.0159
 post  SAL  SR          -0.0719 0.00574 647628  -0.0832  -0.0606
 pre   CIT  SR           0.0000 0.01071 647628  -0.0210   0.0210
 post  CIT  SR          -1.2073 0.00757 647628  -1.2222  -1.1925
 pre   SAL  SIR          0.0000 0.00909 647628  -0.0178   0.0178
 post  SAL  SIR          0.1728 0.00643 647628   0.1602   0.1854
 pre   CIT  SIR          0.0000 0.00834 647628  -0.0163   0.0163
 post  CIT  SIR          0.2123 0.00590 647628   0.2008   0.2239
 pre   SAL  FF           0.0000 0.01514 647628  -0.0297   0.0297
 post  SAL  FF           0.1024 0.01071 647628   0.0814   0.1234
 pre   CIT  FF           0.0000 0.01717 647628  -0.0337   0.0337
 post  CIT  FF          -0.7631 0.01215 647628  -0.7869  -0.7393

Confidence level used: 0.95 


In [58]:
%%R
# PRE POST BY NEURON TYPE
cont <- contrast(em, method="pairwise", by=c("drug", "neuron_type")) 
print(cont %>% rbind())


 drug neuron_type contrast   estimate      SE     df t.ratio p.value
 SAL  SR          pre - post   0.0719 0.00994 647628   7.233  <.0001
 CIT  SR          pre - post   1.2073 0.01312 647628  92.048  <.0001
 SAL  SIR         pre - post  -0.1728 0.01113 647628 -15.525  <.0001
 CIT  SIR         pre - post  -0.2123 0.01022 647628 -20.782  <.0001
 SAL  FF          pre - post  -0.1024 0.01855 647628  -5.523  <.0001
 CIT  FF          pre - post   0.7631 0.02103 647628  36.280  <.0001

P value adjustment: bonferroni method for 6 tests 


In [59]:
%%R
# EFFECT OF DRUG BY NEURON 
cont2 <- pairs(cont, by="neuron_type")
print(cont2 %>% rbind())

# EFFECT OF DRUG ON PRE POST COMPARISON ACROSS NEURONS 
cont3 <- contrast(em, method="pairwise", interaction=T, by=NULL) 
print(cont3 %>% rbind() )

 neuron_type contrast                            estimate     SE     df t.ratio
 SR          (pre - post SAL) - (pre - post CIT)  -1.1354 0.0165 647628 -68.989
 SIR         (pre - post SAL) - (pre - post CIT)   0.0395 0.0151 647628   2.617
 FF          (pre - post SAL) - (pre - post CIT)  -0.8655 0.0280 647628 -30.863
 p.value
  <.0001
  0.0266
  <.0001

P value adjustment: bonferroni method for 3 tests 
 block_pairwise drug_pairwise neuron_type_pairwise estimate     SE     df
 pre - post     SAL - CIT     SR - SIR               -1.175 0.0223 647628
 pre - post     SAL - CIT     SR - FF                -0.270 0.0325 647628
 pre - post     SAL - CIT     SIR - FF                0.905 0.0319 647628
 t.ratio p.value
 -52.593  <.0001
  -8.301  <.0001
  28.412  <.0001

P value adjustment: bonferroni method for 3 tests 


# Way

In [71]:
df_way = load_drug_data(block="way")
df_way["zcounts"] = df_way.groupby("neuron_id")["counts"].transform(lambda x: zmap(x, x[:600]))
df_way = df_way.dropna()

df_responders = (
    df_way
    .groupby("neuron_id")
    .apply(mwu)
    .reset_index()
    .merge(neuron_types)
)
df_responders.to_csv(Config.derived_data_dir / "way_responders.csv", index=False)
df_way.to_csv(Config.derived_data_dir / "way_binned.csv", index=False)

In [72]:
%%R

suppressMessages(require(lme4))
suppressMessages(require(lmerTest))
suppressMessages(require(emmeans))
suppressMessages(require(tidyverse))

p <- "C:\\Users\\roryl\\repos\\DRN Interactions\\data\\derived\\way_binned.csv"

df <- suppressMessages(read_csv(p)) %>%
  mutate(
    neuron_type=factor(neuron_type, levels=c("SR", "SIR", "FF")),
    drug=factor(
      if_else(str_detect(group_name ,"sal"), "SAL", "CIT"), 
      levels=c("SAL", "CIT")),
    block=factor(block, levels=c("pre", "post"))
    )

# Model
mod <- lm(zcounts ~ block * drug *neuron_type, data=df)

# ANOVA
print(anova(mod))

Analysis of Variance Table

Response: zcounts
                           Df  Sum Sq Mean Sq  F value    Pr(>F)    
block                       1    3089    3089  293.777 < 2.2e-16 ***
drug                        1     443     443   42.086 8.741e-11 ***
neuron_type                 2   12578    6289  598.108 < 2.2e-16 ***
block:drug                  1     221     221   21.061 4.451e-06 ***
block:neuron_type           2    6294    3147  299.304 < 2.2e-16 ***
drug:neuron_type            2   74734   37367 3553.738 < 2.2e-16 ***
block:drug:neuron_type      2   37398   18699 1778.351 < 2.2e-16 ***
Residuals              500110 5258548      11                       
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1


In [73]:
%%R

# EMMEANS
em <- emmeans(mod, ~ block*drug*neuron_type)

print(em)


 block drug neuron_type emmean     SE    df lower.CL upper.CL
 pre   SAL  SR           0.000 0.0148 5e+05  -0.0290   0.0290
 post  SAL  SR          -0.057 0.0105 5e+05  -0.0776  -0.0365
 pre   CIT  SR           0.000 0.0255 5e+05  -0.0499   0.0499
 post  CIT  SR           1.894 0.0180 5e+05   1.8589   1.9296
 pre   SAL  SIR          0.000 0.0167 5e+05  -0.0327   0.0327
 post  SAL  SIR          0.439 0.0118 5e+05   0.4160   0.4623
 pre   CIT  SIR          0.000 0.0158 5e+05  -0.0310   0.0310
 post  CIT  SIR         -0.331 0.0112 5e+05  -0.3525  -0.3086
 pre   SAL  FF           0.000 0.0304 5e+05  -0.0595   0.0595
 post  SAL  FF          -0.117 0.0215 5e+05  -0.1587  -0.0745
 pre   CIT  FF           0.000 0.0304 5e+05  -0.0595   0.0595
 post  CIT  FF          -0.134 0.0215 5e+05  -0.1765  -0.0923

Confidence level used: 0.95 


In [74]:
%%R
# PRE POST BY NEURON TYPE
cont <- contrast(em, method="pairwise", by=c("drug", "neuron_type")) 
print(cont %>% rbind())


 drug neuron_type contrast   estimate     SE    df t.ratio p.value
 SAL  SR          pre - post    0.057 0.0181 5e+05   3.147  0.0099
 CIT  SR          pre - post   -1.894 0.0312 5e+05 -60.700  <.0001
 SAL  SIR         pre - post   -0.439 0.0204 5e+05 -21.495  <.0001
 CIT  SIR         pre - post    0.331 0.0194 5e+05  17.054  <.0001
 SAL  FF          pre - post    0.117 0.0372 5e+05   3.135  0.0103
 CIT  FF          pre - post    0.134 0.0372 5e+05   3.613  0.0018

P value adjustment: bonferroni method for 6 tests 


In [75]:
%%R
# EFFECT OF DRUG BY NEURON 
cont2 <- pairs(cont, by="neuron_type")
print(cont2 %>% rbind())

# EFFECT OF DRUG ON PRE POST COMPARISON ACROSS NEURONS 
cont3 <- contrast(em, method="pairwise", interaction=T, by=NULL) 
print(cont3 %>% rbind() )

 neuron_type contrast                            estimate     SE    df t.ratio
 SR          (pre - post SAL) - (pre - post CIT)   1.9513 0.0361 5e+05  54.067
 SIR         (pre - post SAL) - (pre - post CIT)  -0.7697 0.0282 5e+05 -27.331
 FF          (pre - post SAL) - (pre - post CIT)  -0.0178 0.0526 5e+05  -0.338
 p.value
  <.0001
  <.0001
  1.0000

P value adjustment: bonferroni method for 3 tests 
 block_pairwise drug_pairwise neuron_type_pairwise estimate     SE    df
 pre - post     SAL - CIT     SR - SIR                2.721 0.0458 5e+05
 pre - post     SAL - CIT     SR - FF                 1.969 0.0638 5e+05
 pre - post     SAL - CIT     SIR - FF               -0.752 0.0597 5e+05
 t.ratio p.value
  59.439  <.0001
  30.864  <.0001
 -12.600  <.0001

P value adjustment: bonferroni method for 3 tests 
